In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms, models
from tqdm import tqdm 

In [2]:
IMAGE_SIZE = 224

transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:
mncai_dataset = datasets.ImageFolder(root="./data/mncai/train", transform=transform)

val_ratio = 0.2  
test_ratio = 0.1
train_size = int((1 - val_ratio - test_ratio) * len(mncai_dataset))
val_size = int(val_ratio * len(mncai_dataset))
test_size = len(mncai_dataset) - train_size - val_size

mncai_train_dataset, mncai_val_dataset, mncai_test_dataset = random_split(mncai_dataset, [train_size, val_size, test_size])

In [5]:
CIFAKE_train_dataset = datasets.ImageFolder(root="./data/CIFAKE/train", transform=transform)
CIFAKE_test_dataset = datasets.ImageFolder(root="./data/CIFAKE/test", transform=transform)

val_ratio = 0.2  
train_size = int((1 - val_ratio) * len(CIFAKE_train_dataset))
val_size = len(CIFAKE_train_dataset) - train_size

CIFAKE_train_dataset, CIFAKE_val_dataset = random_split(CIFAKE_train_dataset, [train_size, val_size])

In [6]:
BATCH_SIZE = 128

train_dataset = ConcatDataset([mncai_train_dataset, CIFAKE_train_dataset])
val_dataset = ConcatDataset([mncai_val_dataset, CIFAKE_val_dataset])
test_dataset = ConcatDataset([mncai_test_dataset, CIFAKE_test_dataset])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [8]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze earlier layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for binary classification with sigmoid
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1)
)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [9]:
EPOCHS = 100
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0

    # Training
    for images, labels in tqdm(train_loader, desc=f"Epoch (Training) {epoch+1}/{EPOCHS}", leave = False):
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # reshape to (batch, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = torch.sigmoid(outputs) >= 0.5
        correct += (preds == labels.bool()).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)  
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc=f"Epoch (Validation) {epoch+1}/{EPOCHS}", leave = False):
            val_images, val_labels = val_images.to(device), val_labels.to(device).float().unsqueeze(1)
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item() * val_images.size(0)
            val_preds = torch.sigmoid(val_outputs) >= 0.5
            val_correct += (val_preds == val_labels.bool()).sum().item()
            val_total += val_labels.size(0)

    val_acc = val_correct / val_total
    val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)    
    
    # Early stopping condition
    if epoch > 0 and val_loss > val_losses[-2]:
        print(f"Early stopping at epoch {epoch+1}")
        break    
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

torch.save(model, "./models/detection_resnet18.pth")

Epoch (Training) 1/100:   0%|          | 0/735 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.4121, Train Acc: 0.8163, Val Loss: 0.3755, Val Acc: 0.8327


Epoch 2/100, Train Loss: 0.3510, Train Acc: 0.8484, Val Loss: 0.3435, Val Acc: 0.8511


Epoch 3/100, Train Loss: 0.3372, Train Acc: 0.8544, Val Loss: 0.3346, Val Acc: 0.8552


Epoch 4/100, Train Loss: 0.3327, Train Acc: 0.8559, Val Loss: 0.3289, Val Acc: 0.8600


Epoch 5/100, Train Loss: 0.3304, Train Acc: 0.8579, Val Loss: 0.3271, Val Acc: 0.8593


Early stopping at epoch 6


In [10]:
threshold = 0.7 

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs >= threshold).int() 

        correct += (preds == labels.int()).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Test Accuracy (threshold={threshold}): {test_accuracy:.4f}")

Test Accuracy (threshold=0.7): 0.7530
